In [1]:
from pennylane import numpy as np 
import pennylane as qml
from pennylane import qchem
from functools import partial

In [2]:
def info_from_data(data):
    symbols = []
    coordinates = []
    for par in data:
        atom, coordinate = par[0], par[1]
        symbols.append(atom)
        for x in coordinate:
            coordinates.append(x)
    return (symbols, np.array(coordinates))

# symbols = ["H", "H"]
# coordinates = np.array([0.0, 0.0, -0.6614, 0.0, 0.0, 0.6614])

# data = [["H", [0.0, 0.0, -0.6614]], ["H", [0.0, 0.0, 0.6614]]]
data = [["Li", [0, 0, 0]], ["H", [1, 0, 0]]]
symbols, coordinates = info_from_data(data)
print(symbols)
print(coordinates)

['Li', 'H']
[0 0 0 1 0 0]


In [3]:
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates)
print("Number of qubits = ", qubits)
# print(f"The Hamiltonian is {H}")

Number of qubits =  12


In [4]:
dev = qml.device("default.qubit", wires=qubits)

In [5]:
electrons = 4

hf = qml.qchem.hf_state(electrons, qubits)
print(hf)

[1 1 1 1 0 0 0 0 0 0 0 0]


In [7]:
# def circuit(param, wires):
#     qml.BasisState(hf, wires=wires)
#     qml.DoubleExcitation(param, wires=[0, 1, 2, 3])

In [8]:
# @qml.qnode(dev)
# def cost_fn(param):
#     circuit(param, wires=range(qubits))
#     return qml.expval(H)

In [6]:
# Generate single and double excitations
singles, doubles = qchem.excitations(electrons, qubits)

In [7]:
# Map excitations to the wires the UCCSD circuit will act on
s_wires, d_wires = qchem.excitations_to_wires(singles, doubles)

In [8]:
# Define the UCCSD ansatz
ansatz = partial(qml.UCCSD, init_state=hf, s_wires=s_wires, d_wires=d_wires)

# Define the cost function
cost_fn = qml.ExpvalCost(ansatz, H, dev)

In [9]:
params = np.random.normal(0, np.pi, len(singles)+len(doubles))
print(f"len of singles = {len(singles)}, len of doubles = {len(doubles)}")
print(params)
print(cost_fn(params))

len of singles = 16, len of doubles = 76
[ 1.14014682e+00  4.89375883e+00  2.66664485e+00 -2.43130238e+00
 -1.31505995e+00  7.13306410e+00  4.95237368e+00 -3.11610555e+00
 -1.92928042e+00 -1.19794054e+00 -3.03742346e-01  1.84009425e+00
  3.56539545e-01  4.58398326e+00 -1.89499788e+00  1.47658962e-01
  6.64674076e-01  5.41987077e+00 -2.87601131e+00 -5.03565980e+00
 -4.90202696e-01  4.05566769e+00 -4.59020077e+00  4.09591428e+00
  2.52665035e+00  1.34759741e+00 -3.85069354e+00  5.23164986e+00
  6.00089582e-01  3.20268476e+00  3.09864450e+00  3.79444516e+00
  1.80031765e+00 -1.85538413e+00 -3.61836826e+00 -5.57074281e-01
  8.02763890e-01  1.19067234e+00  3.97632855e+00  1.23653556e+00
 -4.59597372e+00 -4.19818650e-02 -1.19579083e+00 -2.34864178e+00
 -3.40457825e+00  1.90128429e+00 -2.33715775e+00  1.99425613e+00
 -6.35638187e+00 -3.14262289e+00 -3.79382373e-03  5.50247685e+00
  9.24658958e-01 -7.18108305e+00 -2.19204827e+00 -7.44171253e+00
  5.20540052e+00  2.00655855e+00 -2.03405416e+00 

KeyboardInterrupt: 

In [22]:
opt = qml.GradientDescentOptimizer(stepsize=0.4)

# theta = np.array(0.0, requires_grad=True)
theta = np.random.normal(0, np.pi, len(singles)+len(doubles), requires_grad=True)


In [25]:
energy = [cost_fn(theta)]

angle = [theta]

max_iterations = 50
conv_tol = 1e-6

for n in range(max_iterations):
    theta, prev_energy = opt.step_and_cost(cost_fn, theta)

    energy.append(cost_fn(theta))
    angle.append(theta)

    conv = np.abs(energy[-1] - prev_energy)

    if n % 5 == 0:
        print(f"Step = {n}, Energy = {energy[-1]:.8f} Ha")

print(f"Final value of the ground-state energy = {energy[-1]:.8f} Ha")
print(f"Optimal value of the circuit parameter = {angle[-1]}")



Step = 0, Energy = -1.13618916 Ha
Step = 5, Energy = -1.13618916 Ha
Final value of the ground-state energy = -1.13618916 Ha
Optimal value of the circuit parameter = [ 3.14158666  3.14159865 -0.20973289]
